In [ ]:
import sys  
sys.path.insert(0, '../build/lib')

import numpy as np
import py_aff3ct as aff3ct
import math

In [ ]:
K = 8
N = 16
ebn0_min = 1.
ebn0_max = 10.

sigma_vals = []
for ebn0 in range(int(ebn0_min), int(ebn0_max)):
    esn0 = ebn0 + 10 * math.log10(K/N)
    sigma_vals.append(math.sqrt(1 / (2 * 10 ** (esn0 / 10))))

In [ ]:
src = aff3ct.module.source.Source_random(K, 12)
enc = aff3ct.module.encoder.Encoder_repetition_sys(K, N)
mdm = aff3ct.module.modem.Modem_BPSK(N)
chn = aff3ct.module.channel.Channel_AWGN_LLR(N)
#dec = aff3ct.module.decoder.Decoder_repetition_std(K, N);
V_K = np.ndarray((1,K), np.int)

In [ ]:
sigma = np.ndarray(shape = (1,1),  dtype = np.float32)
enc[     "encode::U_K" ].bind(src[  "generate::U_K" ])
mdm[   "modulate::X_N1"].bind(enc[    "encode::X_N" ])
chn[  "add_noise::CP"  ].bind(                 sigma )
chn[  "add_noise::X_N" ].bind(mdm[  "modulate::X_N2"])
mdm[ "demodulate::CP"  ].bind(                 sigma )
mdm[ "demodulate::Y_N1"].bind(chn[ "add_noise::Y_N" ])
#dec["decode_siho::Y_N" ].bind(mdm["demodulate::Y_N2"])

In [ ]:
print (sigma_vals)
fer = []
for s in sigma_vals:
    
    
    print("dbg ---------------------------------")
    print (s)
    sigma[:] = s
    print(sigma[:])
    chn["add_noise::CP"].bind(sigma)
    print(chn["add_noise::CP"][:])
    print("dbg ---------------------------------")
    
    e = 0
    f = 0
    while (e < 100):
        src("generate"   ).exec()
        enc("encode"     ).exec()
        mdm("modulate"   ).exec()
        chn("add_noise"  ).exec()
        mdm("demodulate" ).exec()
        #dec("decode_siho").exec()

        # patch: repetition decoder
        V_K_float = np.ndarray(shape=(1,K), dtype=float)
        for k in range(K):
            V_K_float[0][k] = mdm["demodulate::Y_N2"][0][k]
            rep_count = int((N / K)) -1
            for r in range(rep_count):
               V_K_float[0][k] = V_K_float[0][k] + mdm["demodulate::Y_N2"][0][(r+1) * K + k]
        # hard decision
        for k in range(K):
            if V_K_float[0][k] > 0:
                V_K[0][k] = 0
            else:
                V_K[0][k] = 1
        
        # debug
        #print("src[  \"generate::U_K\" ]:") 
        #print(src[  "generate::U_K" ][:])
        #print("enc[    \"encode::X_N\" ]:") 
        #print(enc[    "encode::X_N" ][:])
        #print("mdm[  \"modulate::X_N2\"]:") 
        #print(mdm[  "modulate::X_N2"][:])
        #print("chn[ \"add_noise::Y_N\" ]:") 
        #print(chn[ "add_noise::Y_N" ][:])
        #print("mdm[\"demodulate::Y_N2\"]:") 
        #print(mdm["demodulate::Y_N2"][:])
        #print("V_K:")                       
        #print(V_K)
        #print ("------")
                
        if not (src["generate::U_K"][:] == V_K).all():
            e = e + 1
        f = f + 1
    print(e)
    print(f)
    fer.append(e / f)
    
print(fer)
